<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goroutings-and-Channels" data-toc-modified-id="Goroutings-and-Channels-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Goroutings and Channels</a></span><ul class="toc-item"><li><span><a href="#Goroutines" data-toc-modified-id="Goroutines-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Goroutines</a></span></li><li><span><a href="#Example:-Concurrent-Clock-Server" data-toc-modified-id="Example:-Concurrent-Clock-Server-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Example: Concurrent Clock Server</a></span></li><li><span><a href="#Example:-Concurrent-Echo-Server" data-toc-modified-id="Example:-Concurrent-Echo-Server-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Example: Concurrent Echo Server</a></span></li><li><span><a href="#Channels" data-toc-modified-id="Channels-8.4"><span class="toc-item-num">8.4&nbsp;&nbsp;</span>Channels</a></span><ul class="toc-item"><li><span><a href="#Unbuffered-Channels" data-toc-modified-id="Unbuffered-Channels-8.4.1"><span class="toc-item-num">8.4.1&nbsp;&nbsp;</span>Unbuffered Channels</a></span></li><li><span><a href="#Pipelines" data-toc-modified-id="Pipelines-8.4.2"><span class="toc-item-num">8.4.2&nbsp;&nbsp;</span>Pipelines</a></span></li><li><span><a href="#Unidirectional-Channel-Type" data-toc-modified-id="Unidirectional-Channel-Type-8.4.3"><span class="toc-item-num">8.4.3&nbsp;&nbsp;</span>Unidirectional Channel Type</a></span></li><li><span><a href="#Buffered-Channels" data-toc-modified-id="Buffered-Channels-8.4.4"><span class="toc-item-num">8.4.4&nbsp;&nbsp;</span>Buffered Channels</a></span></li></ul></li><li><span><a href="#Looping-in-Parallel" data-toc-modified-id="Looping-in-Parallel-8.5"><span class="toc-item-num">8.5&nbsp;&nbsp;</span>Looping in Parallel</a></span></li><li><span><a href="#Example:-Concurrent-Web-Crawler" data-toc-modified-id="Example:-Concurrent-Web-Crawler-8.6"><span class="toc-item-num">8.6&nbsp;&nbsp;</span>Example: Concurrent Web Crawler</a></span></li><li><span><a href="#Multiplexing-with-select" data-toc-modified-id="Multiplexing-with-select-8.7"><span class="toc-item-num">8.7&nbsp;&nbsp;</span>Multiplexing with select</a></span></li><li><span><a href="#Example:-Concurrent-Directory-Travsal" data-toc-modified-id="Example:-Concurrent-Directory-Travsal-8.8"><span class="toc-item-num">8.8&nbsp;&nbsp;</span>Example: Concurrent Directory Travsal</a></span></li><li><span><a href="#Cancellation" data-toc-modified-id="Cancellation-8.9"><span class="toc-item-num">8.9&nbsp;&nbsp;</span>Cancellation</a></span></li><li><span><a href="#Example:-Chat-Server" data-toc-modified-id="Example:-Chat-Server-8.10"><span class="toc-item-num">8.10&nbsp;&nbsp;</span>Example: Chat Server</a></span></li></ul></li></ul></div>

# Goroutings and Channels

## Goroutines

In [45]:
//goroutine
import ("fmt";"time")
func f(i int) {
    fmt.Println("Hello, World:",i)
}
%%
for i:=0;i<10;i++ {
    go f(i)
    //f(i)
}
time.Sleep(100* time.Millisecond)

//time.Sleep(1* time.Millisecond)

Hello, World: 6
Hello, World: 1
Hello, World: 9
Hello, World: 3
Hello, World: 2
Hello, World: 7
Hello, World: 8
Hello, World: 4
Hello, World: 5
Hello, World: 0


In [56]:
//gopl.io/ch8/spinner
import "time"
func main(){
    go spinner(100 * time.Millisecond)
    //spinner(100 * time.Millisecond)
    //const n = 35
    const n = 43
    fibN := fib(n) // slow
    fmt.Printf("\rFibonacci(%d) = %d\n", n, fibN)
}

func spinner(delay time.Duration) {
    for {
        for _, r := range `-\|/` {
            fmt.Printf("\r%c",r)
            time.Sleep(delay)
        }
    }
}

func fib(x int) int {
    if (x < 2) {
        return x
    }
    return fib(x-1) + fib(x-2)
}
//main()

Fibonacci(43) = 433494437


## Example: Concurrent Clock Server
* gopl.io/ch8/clock1 : sequential clock server writing the current time to the client one per second
  * handleConn(conn)
* gopl.io/ch8/clock2 : handle connections concurrently
  * go handleConn(conn)

In [57]:
// Clock1 is a TCP server that periodically writes the time.
//package main
import (
	"io"
	"log"
	"net"
	"time"
)

func main() {
	listener, err := net.Listen("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}
	for {
		conn, err := listener.Accept()
		if err != nil {
			log.Print(err) // e.g., connection aborted
			continue
		}
		handleConn(conn) // handle one connection at a time
	}
}

func handleConn(c net.Conn) {
	defer c.Close()
	for {
		_, err := io.WriteString(c, time.Now().Format("15:04:05\n"))
		if err != nil {
			return // e.g., client disconnected
		}
		time.Sleep(1 * time.Second)
	}
}
//main()

signal: interrupt


In [61]:
!nc localhost 8000

21:44:14
21:44:15
21:44:16
21:44:17


signal: interrupt


In [60]:
// Netcat1 is a read-only TCP client.
//package main

import (
	"io"
	"log"
	"net"
	"os"
)

func main() {
	conn, err := net.Dial("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}
	defer conn.Close()
	mustCopy(os.Stdout, conn)
}

func mustCopy(dst io.Writer, src io.Reader) {
	if _, err := io.Copy(dst, src); err != nil {
		log.Fatal(err)
	}
}
//main()
//!-

21:43:31
21:43:32
21:43:44
21:43:45
21:43:46
21:43:47
21:43:48


signal: interrupt


In [62]:
//gopl.io/ch8/clock2
import ("net";"log"; "io"; "time")
func main() {
  listener, err := net.Listen("tcp", "localhost:8000")
  if  err != nil {
    log.Fatal(err)
  }
  for {
    conn, err := listener.Accept()
    if err != nil {
        log.Print(err) // e.g., connection aborted
    }
    go handleConn(conn) // handle connections concurrently
  }
}

func handleConn(c net.Conn) {
	defer c.Close()
	for {
		_, err := io.WriteString(c, time.Now().Format("15:04:05\n"))
		if err != nil {
			return // e.g., client disconnected
		}
		time.Sleep(1 * time.Second)
	}
}
//main()

signal: interrupt


## Example: Concurrent Echo Server
* reverb1
  * go handleConn
  * echo()
* reverb2
  * go handleConn
  * go echo()

In [63]:
// Reverb1 is a TCP server that simulates an echo.
//package main

import (
	"bufio"
	"fmt"
	"log"
	"net"
	"strings"
	"time"
)

//!+
func echo(c net.Conn, shout string, delay time.Duration) {
	fmt.Fprintln(c, "\t", strings.ToUpper(shout))
	time.Sleep(delay)
	fmt.Fprintln(c, "\t", shout)
	time.Sleep(delay)
	fmt.Fprintln(c, "\t", strings.ToLower(shout))
}

func handleConn(c net.Conn) {
	input := bufio.NewScanner(c)
	for input.Scan() {
		echo(c, input.Text(), 1*time.Second)
	}
	// NOTE: ignoring potential errors from input.Err()
	c.Close()
}



In [64]:
//go handleConn(conn)
func main() {
	l, err := net.Listen("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}
	for {
		conn, err := l.Accept()
		if err != nil {
			log.Print(err) // e.g., connection aborted
			continue
		}
		go handleConn(conn)
	}
}


^C
signal: interrupt


In [ ]:
//main()

In [67]:
//gopl.io/ch8/netcat2
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 223.

// Netcat is a simple read/write client for TCP servers.
//package main

import (
	"io"
	"log"
	"net"
	"os"
)

//!+
func main() {
	conn, err := net.Dial("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}
	defer conn.Close()
	go mustCopy(os.Stdout, conn)
	mustCopy(conn, os.Stdin)
}

//!-

func mustCopy(dst io.Writer, src io.Reader) {
	if _, err := io.Copy(dst, src); err != nil {
		log.Fatal(err)
	}
}
//main()

21:59:10
21:59:11
21:59:12
21:59:13
21:59:14
21:59:15
21:59:16
21:59:17
21:59:18
21:59:19
21:59:20
21:59:21
21:59:22


^C
signal: interrupt


In [79]:
// Reverb2 is a TCP server that simulates an echo.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 224.

// Reverb2 is a TCP server that simulates an echo.
//package main

import (
	"bufio"
	"fmt"
	"log"
	"net"
	"strings"
	"time"
)

func echo(c net.Conn, shout string, delay time.Duration) {
	fmt.Fprintln(c, "\t", strings.ToUpper(shout))
	time.Sleep(delay)
	fmt.Fprintln(c, "\t", shout)
	time.Sleep(delay)
	fmt.Fprintln(c, "\t", strings.ToLower(shout))
}

//!+
func handleConn(c net.Conn) {
	input := bufio.NewScanner(c)
	for input.Scan() {
		go echo(c, input.Text(), 1*time.Second)
	}
	// NOTE: ignoring potential errors from input.Err()
	c.Close()
}



In [80]:
// go handleConn()
import ("net"
        "log"
       )
func main() {
	l, err := net.Listen("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}
	for {
		conn, err := l.Accept()
		if err != nil {
			log.Print(err) // e.g., connection aborted
			continue
		}
		go handleConn(conn)
	}
}


^C
signal: interrupt


In [ ]:
//main()

## Channels

In [81]:
import "fmt"
%%
ch := make(chan int) // ch has type 'chan int'
go func() {ch <- 9}()
//ch <- 9
x := <- ch
fmt.Println(x)
close(ch) 

9


### Unbuffered Channels

In [74]:
//gopl.io/ch8/netcat3
// Netcat is a simple read/write client for TCP servers.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 227.


//package main

import (
	"io"
	"log"
	"net"
	"os"
)

//!+
func main() {
	conn, err := net.Dial("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}
	done := make(chan struct{})
	go func() {
		io.Copy(os.Stdout, conn) // NOTE: ignoring errors
		log.Println("done")
		done <- struct{}{} // signal the main goroutine
	}()
	mustCopy(conn, os.Stdin)
	conn.Close()
	<-done // wait for background goroutine to finish
}

//!-

func mustCopy(dst io.Writer, src io.Reader) {
	if _, err := io.Copy(dst, src); err != nil {
		log.Fatal(err)
	}
}
//main()

22:09:58
22:09:59
22:10:00
22:10:01
22:10:02
22:10:03
22:10:04
22:10:05
22:10:06
22:10:07
22:10:08
22:10:09
22:10:10
22:10:11
22:10:12
22:10:13
22:10:14
09:58:09
09:58:10
09:58:11
09:58:12
09:58:13
09:58:14
09:58:15
09:58:16
09:58:17
09:58:18
09:58:19
09:58:20
09:58:21
09:58:22
09:58:23
09:58:24
09:58:25
09:58:26
09:58:27
09:58:28
09:58:29
09:58:30
09:58:31
09:58:32
09:58:33
09:58:34
09:58:35
09:58:36
09:58:37
09:58:38
09:58:39
09:58:40
09:58:41
09:58:42
09:58:43
09:58:44
09:58:45
09:58:46
09:58:47
09:58:48
09:58:49
09:58:50
09:58:51
09:58:52
09:58:53
09:58:54
09:58:55
09:58:56
09:58:57
09:58:58
09:58:59
09:59:00
09:59:01
09:59:02
09:59:03
09:59:04
09:59:05
09:59:06
09:59:07
09:59:08
09:59:09


^C
signal: interrupt


### Pipelines

In [82]:
// Pipeline1 demonstrates an infinite 3-stage pipeline.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 228.


//package main

import "fmt"

//!+
func main() {
	naturals := make(chan int)
	squares := make(chan int)

	// Counter
	go func() {
		for x := 0; x<100; x++ {
			naturals <- x
		}
        //close(naturals)
	}()

	// Squarer
	go func() {
		for {
			x := <-naturals
			squares <- x * x
		}
	}()

	// Printer (in main goroutine)
	for {
		fmt.Println(<-squares)
	}
}

//main()
//!-

0
1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400
441
484
529
576
625
676
729
784
841
900
961
1024
1089
1156
1225
1296
1369
1444
1521
1600
1681
1764
1849
1936
2025
2116
2209
2304
2401
2500
2601
2704
2809
2916
3025
3136
3249
3364
3481
3600
3721
3844
3969
4096
4225
4356
4489
4624
4761
4900
5041
5184
5329
5476
5625
5776
5929
6084
6241
6400
6561
6724
6889
7056
7225
7396
7569
7744
7921
8100
8281
8464
8649
8836
9025
9216
9409
9604
9801


^C
signal: interrupt


In [83]:
// Pipeline2 demonstrates a finite 3-stage pipeline.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 229.


//package main

import "fmt"

//!+
func main() {
	naturals := make(chan int)
	squares := make(chan int)

	// Counter
	go func() {
		for x := 0; x < 100; x++ {
			naturals <- x
		}
		close(naturals)
	}()

	// Squarer
	go func() {
		for x := range naturals {
			squares <- x * x
		}
		close(squares)
	}()

	// Printer (in main goroutine)
	for x := range squares {
		fmt.Println(x)
	}
}

//main()
//!-

0
1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400
441
484
529
576
625
676
729
784
841
900
961
1024
1089
1156
1225
1296
1369
1444
1521
1600
1681
1764
1849
1936
2025
2116
2209
2304
2401
2500
2601
2704
2809
2916
3025
3136
3249
3364
3481
3600
3721
3844
3969
4096
4225
4356
4489
4624
4761
4900
5041
5184
5329
5476
5625
5776
5929
6084
6241
6400
6561
6724
6889
7056
7225
7396
7569
7744
7921
8100
8281
8464
8649
8836
9025
9216
9409
9604
9801


### Unidirectional Channel Type

In [84]:
// Pipeline3 demonstrates a finite 3-stage pipeline
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 231.


// with range, close, and unidirectional channel types.
//package main

import "fmt"

//!+
func counter(out chan<- int) {
	for x := 0; x < 100; x++ {
		out <- x
	}
	close(out)
}

func squarer(out chan<- int, in <-chan int) {
	for v := range in {
		out <- v * v
	}
	close(out)
}

func printer(in <-chan int) {
	for v := range in {
		fmt.Println(v)
	}
}

func main() {
	naturals := make(chan int)
	squares := make(chan int)

	go counter(naturals)
	go squarer(squares, naturals)
	printer(squares)
}

//!-

0
1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400
441
484
529
576
625
676
729
784
841
900
961
1024
1089
1156
1225
1296
1369
1444
1521
1600
1681
1764
1849
1936
2025
2116
2209
2304
2401
2500
2601
2704
2809
2916
3025
3136
3249
3364
3481
3600
3721
3844
3969
4096
4225
4356
4489
4624
4761
4900
5041
5184
5329
5476
5625
5776
5929
6084
6241
6400
6561
6724
6889
7056
7225
7396
7569
7744
7921
8100
8281
8464
8649
8836
9025
9216
9409
9604
9801


In [3]:
//main()

0
1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400
441
484
529
576
625
676
729
784
841
900
961
1024
1089
1156
1225
1296
1369
1444
1521
1600
1681
1764
1849
1936
2025
2116
2209
2304
2401
2500
2601
2704
2809
2916
3025
3136
3249
3364
3481
3600
3721
3844
3969
4096
4225
4356
4489
4624
4761
4900
5041
5184
5329
5476
5625
5776
5929
6084
6241
6400
6561
6724
6889
7056
7225
7396
7569
7744
7921
8100
8281
8464
8649
8836
9025
9216
9409
9604
9801


### Buffered Channels

In [91]:
var ch chan string
func chinit() {
    ch = make(chan string, 3)
    ch <- "A"
    ch <- "B"
    ch <- "C"
}


In [95]:
%%
chinit()
fmt.Println(<- ch) // "A"
fmt.Println(cap(ch)) // "3"
fmt.Println(len(ch)) // "2"
fmt.Println(<- ch) // "B"
fmt.Println(<- ch) // "C"

A
3
2
B
C


In [96]:
func mirroredQuery() string {
    responses := make(chan string,3)
    go func() { responses <- request("asia.gopl.io")}()
    go func() { responses <- request("europe.gopl.io")}()
    go func() { responses <- request("americas.gopl.io")}()
    return <- responses // return the quickest response
}

# gonb_5e939c0a 

 

 Cell[96]: Line 3 
 ./main.go:62:30: undefined: request
 

func mirroredQuery() string {
 responses := make(chan string,3)
 go func() { responses <- request("asia.gopl.io")}()
 go func() { responses <- request("europe.gopl.io")}()
 go func() { responses <- request("americas.gopl.io")}()

 

 

 Cell[96]: Line 4 
 ./main.go:63:30: undefined: request
 
func mirroredQuery() string {
 responses := make(chan string,3)
 go func() { responses <- request("asia.gopl.io")}()
 go func() { responses <- request("europe.gopl.io")}()
 go func() { responses <- request("americas.gopl.io")}()
 return <- responses // return the quickest response

 

 

 Cell[96]: Line 5 
 ./main.go:64:30: undefined: request
 
 responses := make(chan string,3)
 go func() { responses <- request("asia.gopl.io")}()
 go func() { responses <- request("europe.gopl.io")}()
 go func() { responses <- request("americas.gopl.io")}()
 return <- responses // return the quickest response
}

ERROR: failed to run "/opt/homebrew/bin/go build -o /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_5e939c0a/gonb_5e939c0a": exit status 1

## Looping in Parallel

In [97]:
//gopl.io/ch8/thumbnail
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 234.

// The thumbnail package produces thumbnail-size images from
// larger images.  Only JPEG images are currently supported.
//package thumbnail

import (
	"fmt"
	"image"
	"image/jpeg"
	"io"
	"os"
	"path/filepath"
	"strings"
)

// Image returns a thumbnail-size version of src.
func Image(src image.Image) image.Image {
	// Compute thumbnail size, preserving aspect ratio.
	xs := src.Bounds().Size().X
	ys := src.Bounds().Size().Y
	width, height := 128, 128
	if aspect := float64(xs) / float64(ys); aspect < 1.0 {
		width = int(128 * aspect) // portrait
	} else {
		height = int(128 / aspect) // landscape
	}
	xscale := float64(xs) / float64(width)
	yscale := float64(ys) / float64(height)

	dst := image.NewRGBA(image.Rect(0, 0, width, height))

	// a very crude scaling algorithm
	for x := 0; x < width; x++ {
		for y := 0; y < height; y++ {
			srcx := int(float64(x) * xscale)
			srcy := int(float64(y) * yscale)
			dst.Set(x, y, src.At(srcx, srcy))
		}
	}
	return dst
}

// ImageStream reads an image from r and
// writes a thumbnail-size version of it to w.
func ImageStream(w io.Writer, r io.Reader) error {
	src, _, err := image.Decode(r)
	if err != nil {
		return err
	}
	dst := Image(src)
	return jpeg.Encode(w, dst, nil)
}

// ImageFile2 reads an image from infile and writes
// a thumbnail-size version of it to outfile.
func ImageFile2(outfile, infile string) (err error) {
	in, err := os.Open(infile)
	if err != nil {
		return err
	}
	defer in.Close()

	out, err := os.Create(outfile)
	if err != nil {
		return err
	}

	if err := ImageStream(out, in); err != nil {
		out.Close()
		return fmt.Errorf("scaling %s to %s: %s", infile, outfile, err)
	}
	return out.Close()
}

// ImageFile reads an image from infile and writes
// a thumbnail-size version of it in the same directory.
// It returns the generated file name, e.g. "foo.thumb.jpeg".
func ImageFile(infile string) (string, error) {
	ext := filepath.Ext(infile) // e.g., ".jpg", ".JPEG"
	outfile := strings.TrimSuffix(infile, ext) + ".thumb" + ext
	return outfile, ImageFile2(outfile, infile)
}

In [98]:
// gopl.io/ch8/thumbnail_test
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// This file is just a place to put example code from the book.
// It does not actually run any code in gopl.io/ch8/thumbnail.

//package thumbnail_test

import (
	"log"
	"os"
	"sync"

	"gopl.io/ch8/thumbnail"
)

//!+1
// makeThumbnails makes thumbnails of the specified files.
func makeThumbnails(filenames []string) {
	for _, f := range filenames {
		if _, err := thumbnail.ImageFile(f); err != nil {
			log.Println(err)
		}
	}
}



In [99]:
// makeThumbnails2
// NOTE: incorrect!
func makeThumbnails2(filenames []string) {
	for _, f := range filenames {
		go thumbnail.ImageFile(f) // NOTE: ignoring errors
	}
}

In [100]:
// makeThumbnails3 makes thumbnails of the specified files in parallel.
func makeThumbnails3(filenames []string) {
	ch := make(chan struct{})
	for _, f := range filenames {
		go func(f string) {
			thumbnail.ImageFile(f) // NOTE: ignoring errors
			ch <- struct{}{}
		}(f)
	}

	// Wait for goroutines to complete.
	for range filenames {
		<-ch
	}
}


In [101]:
// makeThumbnails4 makes thumbnails for the specified files in parallel.
// It returns an error if any step failed.
func makeThumbnails4(filenames []string) error {
	errors := make(chan error)

	for _, f := range filenames {
		go func(f string) {
			_, err := thumbnail.ImageFile(f)
			errors <- err
		}(f)
	}

	for range filenames {
		if err := <-errors; err != nil {
			return err // NOTE: incorrect: goroutine leak!
		}
	}

	return nil
}

In [102]:
// makeThumbnails5 makes thumbnails for the specified files in parallel.
// It returns the generated file names in an arbitrary order,
// or an error if any step failed.
func makeThumbnails5(filenames []string) (thumbfiles []string, err error) {
	type item struct {
		thumbfile string
		err       error
	}

	ch := make(chan item, len(filenames))
	for _, f := range filenames {
		go func(f string) {
			var it item
			it.thumbfile, it.err = thumbnail.ImageFile(f)
			ch <- it
		}(f)
	}

	for range filenames {
		it := <-ch
		if it.err != nil {
			return nil, it.err
		}
		thumbfiles = append(thumbfiles, it.thumbfile)
	}

	return thumbfiles, nil
}


In [103]:
// makeThumbnails6 makes thumbnails for each file received from the channel.
// It returns the number of bytes occupied by the files it creates.
func makeThumbnails6(filenames <-chan string) int64 {
	sizes := make(chan int64)
	var wg sync.WaitGroup // number of working goroutines
	for f := range filenames {
		wg.Add(1)
		// worker
		go func(f string) {
			defer wg.Done()
			thumb, err := thumbnail.ImageFile(f)
			if err != nil {
				log.Println(err)
				return
			}
			info, _ := os.Stat(thumb) // OK to ignore error
			sizes <- info.Size()
		}(f)
	}

	// closer
	go func() {
		wg.Wait()
		close(sizes)
	}()

	var total int64
	for size := range sizes {
		total += size
	}
	return total
}

## Example: Concurrent Web Crawler

* A simple web crawler exploring the link graph in BFS
* concurrent and independent calls to crawl 
* exploit the I/O parallelism
  * crawl1 : unbounded parallelism
  * crawl2 : limited parallelism with counting semaphore
  * crawl3 : at most 20 HTTP reequests active concurrently

In [105]:
// Crawl1 crawls web links starting with the command-line arguments.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 240.


//
// This version quickly exhausts available file descriptors
// due to excessive concurrent calls to links.Extract.
//
// Also, it never terminates because the worklist is never closed.
//package main

import (
	"fmt"
	"log"
	"os"

	"gopl.io/ch5/links"
)

//!+crawl
func crawl(url string) []string {
	fmt.Println(url)
	list, err := links.Extract(url)
	if err != nil {
		log.Print(err)
	}
	return list
}

//!-crawl

//!+main
%args http://gopl.io/
func main() {
	worklist := make(chan []string)

	// Start with the command-line arguments.
	go func() { worklist <- os.Args[1:] }()

	// Crawl the web concurrently.
	seen := make(map[string]bool)
	for list := range worklist {
		for _, link := range list {
			if !seen[link] {
				seen[link] = true
				go func(link string) {
					worklist <- crawl(link)
				}(link)
			}
		}
	}
}

//!-main

/*
//!+output
$ go build gopl.io/ch8/crawl1
$ ./crawl1 http://gopl.io/
http://gopl.io/
https://golang.org/help/

https://golang.org/doc/
https://golang.org/blog/
...
2015/07/15 18:22:12 Get ...: dial tcp: lookup blog.golang.org: no such host
2015/07/15 18:22:12 Get ...: dial tcp 23.21.222.120:443: socket:
                                                        too many open files
...
//!-output
*/

http://gopl.io/
http://www.informit.com/store/go-programming-language-9780134190440
http://www.amazon.com/dp/0134190440
http://www.gopl.io/errata.html
http://www.barnesandnoble.com/w/1121601944
http://www.amazon.com/dp/020161586X?tracking_id=disfordig-20
https://github.com/adonovan/gopl.io/
http://www.gopl.io/reviews.html
http://www.gopl.io/ch1.pdf
http://www.gopl.io/translations.html
http://golang.org/s/oracle-user-manual
https://github.com/golang/tools/blob/master/refactor/eg/eg.go
http://golang.org/lib/godoc/analysis/help.html
https://github.com/golang/tools/blob/master/refactor/rename/rename.go
http://www.amazon.com/dp/0131103628?tracking_id=disfordig-20
https://github.com/golang/proposal/blob/master/design/12416-cgo-pointers.md
http://www.amazon.com/Programming-Language-Addison-Wesley-Professional-Computing/product-reviews/0134190440/ref=cm_cr_dp_see_all_summary
http://eli.thegreenplace.net/2016/book-review-the-go-programming-language-by-alan-donovan-and-brian-kernighan
https://ww

https://www.tenlong.com.tw/products/9780134190440?list_name=trs-f
https://www.tenlong.com.tw/products/9789864347551?list_name=rd
https://www.tenlong.com.tw/categories/computer-vision
https://www.tenlong.com.tw/events/2023-bts-booklist
https://www.tenlong.com.tw/products/9789864762637?list_name=rd
https://www.tenlong.com.tw/categories/computer-networks
https://www.tenlong.com.tw/products/9789863126706?list_name=sp
https://www.tenlong.com.tw/products/9787115445353?list_name=rd
https://www.tenlong.com.tw/categories/semicondutor
https://www.tenlong.com.tw/categories/bootstrap
https://www.tenlong.com.tw/products/9487000721684?list_name=rd
https://www.tenlong.com.tw/announcements/310
https://www.tenlong.com.tw/privacy
https://www.tenlong.com.tw/lists/944
https://www.tenlong.com.tw/categories/cross-platform
https://www.tenlong.com.tw/products/9789862017050?list_name=rd
https://www.tenlong.com.tw/products/9787111720645?list_name=sp
https://www.tenlong.com.tw/categories/cmos
https://www.tenlong

2023/11/26 16:14:57 getting https://www.tenlong.com.tw/products/9787111590316?list_name=rd: 503 Service Unavailable
2023/11/26 16:14:57 getting https://www.tenlong.com.tw/products/9789864762521?list_name=rd: 503 Service Unavailable
2023/11/26 16:14:57 getting https://www.tenlong.com.tw/categories/mongodb: 503 Service Unavailable
2023/11/26 16:14:57 getting https://www.tenlong.com.tw/categories/laravel: 503 Service Unavailable
2023/11/26 16:14:57 getting https://www.tenlong.com.tw/categories/software-architecture: 503 Service Unavailable
2023/11/26 16:14:57 getting https://www.tenlong.com.tw/categories/wordpress: 503 Service Unavailable
2023/11/26 16:14:57 getting https://www.tenlong.com.tw/products/9789864341405?list_name=rd: 503 Service Unavailable
2023/11/26 16:14:57 getting https://www.tenlong.com.tw/categories/angular: 503 Service Unavailable
2023/11/26 16:14:57 getting https://www.tenlong.com.tw/categories/digital-image: 503 Service Unavailable
2023/11/26 16:14:57 getting https://

2023/11/26 16:14:57 getting https://www.tenlong.com.tw/announcements/313: 503 Service Unavailable
2023/11/26 16:14:57 getting https://www.tenlong.com.tw/categories/data-science: 503 Service Unavailable
2023/11/26 16:14:57 getting https://www.tenlong.com.tw/categories/react: 503 Service Unavailable
2023/11/26 16:14:57 getting https://www.tenlong.com.tw/products/9787111642992?list_name=sp: 503 Service Unavailable
2023/11/26 16:14:57 getting https://www.tenlong.com.tw/categories/flutter: 503 Service Unavailable


https://www.gotop.com.tw/books/language.aspx
https://pay.ecpay.com.tw/CreditPayment/FromCredit?c=3013113&installment=0&payOff=0
https://www.gotop.com.tw/certification/
http://www.evertop.com.tw/
https://www.gotop.com.tw/books/contact01.aspx
https://www.gotop.com.tw/books/
http://books.gotop.com.tw
https://www.gotop.com.tw/certification/Certiport/
https://www.gotop.com.tw/books/oreilly.aspx
https://www.gotop.com.tw/autodesk


...

https://ml.wikipedia.org/wiki/%E0%B4%A8%E0%B5%8D%E0%B4%AF%E0%B4%BE%E0%B4%AF%E0%B5%8B%E0%B4%AA%E0%B4%AF%E0%B5%8B%E0%B4%97%E0%B4%82
https://ms.wikipedia.org/wiki/Kegunaan_wajar
https://en.wikipedia.org/wiki/Property
https://en.wikipedia.org/wiki/Inline_linking
https://en.wikipedia.org/wiki/Property_law
https://en.wikipedia.org/wiki/Fair_use#cite_note-14
https://en.wikipedia.org/wiki/Template:Intellectual_property
https://en.wikipedia.org/wiki/Sony_Corp._of_America_v._Universal_City_Studios,_Inc.
https://en.wikipedia.org/wiki/Template_talk:Intellectual_property
https://en.wikipedia.org/wiki/2_Live_Crew
https://en.wikipedia.org/wiki/Special:EditPage/Template:Intellectual_property
https://my.wikipedia.org/wiki/%E1%80%9E%E1%80%84%E1%80%B7%E1%80%BA%E1%80%9C%E1%80%BB%E1%80%B1%E1%80%AC%E1%80%BA%E1%80%99%E1%80%BB%E1%80%BE%E1%80%90%E1%80%9E%E1%80%8A%E1%80%B7%E1%80%BA_%E1%80%A1%E1%80%9E%E1%80%AF%E1%80%B6%E1%80%B8%E1%80%95%E1%80%BC%E1%80%AF%E1%80%99%E1%80%BE%E1%80%AF
https://en.wikipedia.org/wiki/L

https://en.wikipedia.org/wiki/Fair_use#cite_note-24
https://en.wikipedia.org/wiki/Mattel
https://en.wikipedia.org/wiki/Moral_rights_(copyright_law)
https://en.wikipedia.org/wiki/Fair_use#cite_note-Aufderheide-45
https://en.wikipedia.org/wiki/Talk:Fair_use
https://en.wikipedia.org/w/index.php?title=Fair_use&action=edit&section=5
https://en.wikipedia.org/wiki/Sampling_(music)
https://en.wikipedia.org/wiki/Legal_issues_surrounding_music_sampling
https://en.wikipedia.org/wiki/Biz_Markie
https://ur.wikipedia.org/wiki/%D9%85%D9%86%D8%B5%D9%81%D8%A7%D9%86%DB%81_%D8%A7%D8%B3%D8%AA%D8%B9%D9%85%D8%A7%D9%84
https://en.wikipedia.org/wiki/File:GoogleImageSearch.png
https://en.wikipedia.org/w/index.php?title=Fair_use&action=edit
https://en.wikipedia.org/w/index.php?title=Fair_use&action=history
https://en.wikipedia.org/wiki/Time-shifting
https://en.wikipedia.org/wiki/Gilbert_O%27Sullivan
https://en.wikipedia.org/wiki/United_States_Court_of_Appeals_for_the_Ninth_Circuit
https://en.wikipedia.org/wiki/

https://advance.lexis.com/api/permalink/92843008-66b0-412b-bd2c-025480a70e86/?context=1000516
https://en.wikipedia.org/wiki/Fair_use#cite_note-71
https://en.wikipedia.org/wiki/Fair_use#cite_ref-Gyles_2-0
https://en.wikipedia.org/wiki/Fair_use#cite_ref-Gyles_2-1
https://arstechnica.com/tech-policy/news/2009/05/harvard-prof-tells-judge-that-p2p-filesharing-is-fair-use.ars
http://copy.law.cam.ac.uk/cam/pdf/uk_1741_1.pdf
https://en.wikipedia.org/wiki/Fair_use#cite_ref-34
https://en.wikipedia.org/wiki/Fair_use#cite_ref-3
https://en.wikipedia.org/wiki/Fair_use#cite_ref-:0_4-0
https://arstechnica.com/tech-policy/news/2009/05/harvard-law-prof-helping-in-not-1-but-3-file-sharing-cases.ars
https://en.wikipedia.org/wiki/Fair_use#cite_ref-Folsom_5-0
https://en.wikipedia.org/wiki/Fair_use#cite_ref-35
https://en.wikipedia.org/wiki/Fair_use#cite_ref-Folsom_5-1
https://uz.wikipedia.org/wiki/Adolatli_foydalanish
https://advance.lexis.com/api/permalink/9c591014-7c80-4e52-9426-014d2e775f0c/?context=10005

http://www.tc.umn.edu/~nasims/HathivAG10_10_12.pdf
https://en.wikipedia.org/wiki/Fair_use#cite_ref-83
https://en.wikipedia.org/wiki/List_of_parties_to_international_treaties_protecting_rights_related_to_copyright
https://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.196.423
https://web.archive.org/web/20060914153848/http://www.gmu.edu/departments/law//faculty/papers/docs/01-03.pdf
https://en.wikipedia.org/w/index.php?title=Copyright_law_of_Benin&action=edit&redlink=1
https://en.wikipedia.org/wiki/Creative_Commons
https://en.wikipedia.org/wiki/Association_of_Research_Libraries
https://doi.org/10.2307%2F1122296
https://en.wikipedia.org/wiki/Wendy_Gordon
https://en.wikipedia.org/wiki/Copyright_law_of_Burundi
https://en.wikipedia.org/wiki/Fair_use#cite_ref-HarvardFUW_82-0
https://hdl.handle.net/2144%2F22971
https://en.wikipedia.org/wiki/Harvard_University
https://api.semanticscholar.org/CorpusID:151080880
https://en.wikipedia.org/w/index.php?title=Fair_use&action=edit&section=35
https:/

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



* Bracketed links.Extract by operations to acquire and release a token
* Ensuring at most 20 calls to it are active at one time

In [106]:
// Crawl2 crawls web links starting with the command-line arguments.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 241.


//
// This version uses a buffered channel as a counting semaphore
// to limit the number of concurrent calls to links.Extract.
//package main

import (
	"fmt"
	"log"
	"os"

	"gopl.io/ch5/links"
)

//!+sema
// tokens is a counting semaphore used to
// enforce a limit of 20 concurrent requests.
var tokens = make(chan struct{}, 20)

func crawl(url string) []string {
	fmt.Println(url)
	tokens <- struct{}{} // acquire a token
	list, err := links.Extract(url)
	<-tokens // release the token

	if err != nil {
		log.Print(err)
	}
	return list
}

//!-sema

//!+
%args http://gopl.io/
func main() {
	worklist := make(chan []string)
	var n int // number of pending sends to worklist

	// Start with the command-line arguments.
	n++
	go func() { worklist <- os.Args[1:] }()

	// Crawl the web concurrently.
	seen := make(map[string]bool)
	for ; n > 0; n-- {
		list := <-worklist
		for _, link := range list {
			if !seen[link] {
				seen[link] = true
				n++
				go func(link string) {
					worklist <- crawl(link)
				}(link)
			}
		}
	}
}

//!-

http://gopl.io/
http://www.amazon.com/dp/020161586X?tracking_id=disfordig-20
http://www.gopl.io/translations.html
http://www.informit.com/store/go-programming-language-9780134190440
http://www.gopl.io/ch1.pdf
http://golang.org/lib/godoc/analysis/help.html
http://www.gopl.io/errata.html
https://github.com/golang/tools/blob/master/refactor/eg/eg.go
http://www.amazon.com/dp/0134190440
http://www.barnesandnoble.com/w/1121601944
https://github.com/adonovan/gopl.io/
http://www.gopl.io/reviews.html
https://github.com/golang/tools/blob/master/refactor/rename/rename.go
http://golang.org/s/oracle-user-manual
http://www.amazon.com/dp/0131103628?tracking_id=disfordig-20


2023/11/26 16:15:45 getting https://github.com/golang/tools/blob/master/refactor/rename/rename.go: 429 Too Many Requests
2023/11/26 16:15:45 getting https://github.com/golang/tools/blob/master/refactor/eg/eg.go: 429 Too Many Requests
2023/11/26 16:15:45 getting https://github.com/adonovan/gopl.io/: 429 Too Many Requests


https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496
https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou?ie=UTF8&nodeId=508088
http://helion.pl/ksiazki/jezyk-go-poznaj-i-programuj-alan-a-a-donovan-brian-w-kernighan,jgopop.htm
http://helion.pl/
http://www.pearsonapac.com/
http://novatec.com.br/
http://www.gotop.com.tw/
http://www.maruzen.co.jp/corp/en/services/publishing.html
http://www.acornpub.co.kr/book/go-programming
https://www.amazon.cn/dp/B072LCX9S7
http://www.williamspublishing.com/Books/978-5-8459-2051-5.html
https://novatec.com.br/livros/linguagem-de-programacao-go/
http://www.amazon.co.jp/exec/obidos/ASIN/4621300253
https://www.tenlong.com.tw/products/9789864761333


2023/11/26 16:15:45 Get "http://www.pearsonapac.com/": dial tcp: lookup www.pearsonapac.com: no such host


https://pay.ecpay.com.tw/CreditPayment/FromCredit?c=3013113&installment=0&payOff=0
https://www.gotop.com.tw/books/language.aspx
http://www.evertop.com.tw/
https://www.gotop.com.tw/certification/
http://books.gotop.com.tw
https://www.gotop.com.tw/books/oreilly.aspx
https://www.gotop.com.tw/autodesk
https://www.gotop.com.tw/certification/Certiport/
https://www.gotop.com.tw/books/contact01.aspx
https://www.gotop.com.tw/books/
http://www.amazon.com/Programming-Language-Addison-Wesley-Professional-Computing/product-reviews/0134190440/ref=cm_cr_dp_see_all_summary
http://www.computingreviews.com/index_dynamic.cfm?CFID=15675338&CFTOKEN=37047869
https://www.usenix.org/system/files/login/articles/login_dec15_17_books.pdf
http://lpar.ath0.com/2015/12/03/review-go-programming-language-book
http://www.onebigfluke.com/2016/03/book-review-go-programming-language.html
http://www.infoq.com/articles/the-go-programming-language-book-review
http://eli.thegreenplace.net/2016/book-review-the-go-programming-

...

https://www.tenlong.com.tw/publishers/8?page=5
https://www.tenlong.com.tw/publishers/8?sort=date
https://www.tenlong.com.tw/publishers/8?sort=bestselling
https://www.tenlong.com.tw/publishers/8?stock=available
https://www.tenlong.com.tw/publishers/8?stock=buyable
https://www.tenlong.com.tw/products/9789863127666?list_name=m-10
https://www.tenlong.com.tw/products/9789863127727?list_name=m-10
https://www.tenlong.com.tw/products/9789863127604?list_name=m-10
https://www.tenlong.com.tw/products/9789863127710?list_name=m-10
https://www.tenlong.com.tw/products/9789863127703?list_name=m-10
https://www.tenlong.com.tw/products/9789863127680?list_name=m-10
https://www.tenlong.com.tw/products/9789863127642?list_name=m-10
https://www.tenlong.com.tw/products/9789863127109?list_name=m-10
https://www.tenlong.com.tw/products/9789863127628?list_name=m-10
https://www.tenlong.com.tw/products/9789863127659?list_name=m-10
https://www.tenlong.com.tw/products/9789863127543?list_name=m-10
https://www.tenlong.c

https://www.tenlong.com.tw/products/9789572139233?list_name=sp
https://www.tenlong.com.tw/products/9789864309191?list_name=sp
https://www.tenlong.com.tw/products/9789862761663?list_name=sp
https://www.tenlong.com.tw/products/9781491948927?list_name=trs-f
https://www.tenlong.com.tw/products/9789866072116?list_name=rd
https://www.tenlong.com.tw/products/9789864762514?list_name=rd
https://www.tenlong.com.tw/products/9789864762453?list_name=rd
https://www.tenlong.com.tw/products/9789864341801?list_name=rd
https://www.tenlong.com.tw/products/9789864762422?list_name=rd
https://www.tenlong.com.tw/products/9789864762651?list_name=rd
https://www.tenlong.com.tw/products/9789864764242?list_name=sp
https://www.tenlong.com.tw/products/9789864764259?list_name=sp
https://www.tenlong.com.tw/products/9789864765973?list_name=sp
https://www.tenlong.com.tw/products/9789865501754?list_name=sp
https://www.tenlong.com.tw/products/9786263333512?list_name=sp
https://www.tenlong.com.tw/products/9787302180753?li

https://www.tenlong.com.tw/categories/children?page=4
https://www.tenlong.com.tw/products/9789863794097?list_name=rd
https://www.tenlong.com.tw/products/9789864764860?list_name=rd
https://www.tenlong.com.tw/products/9789864768226?list_name=rd
https://www.tenlong.com.tw/products/9787115481184?list_name=sp
https://www.tenlong.com.tw/products/9781491919712?list_name=trs-f
https://www.tenlong.com.tw/categories/children?page=5
https://www.tenlong.com.tw/products/9789863478652?list_name=rd
https://www.tenlong.com.tw/products/9789864340958?list_name=rd
https://www.tenlong.com.tw/products/9789864340347?list_name=sp
https://www.tenlong.com.tw/products/9787121298042?list_name=sp
https://www.tenlong.com.tw/products/9789865027810?list_name=sp
https://www.tenlong.com.tw/products/9786263333086?list_name=sp
https://www.tenlong.com.tw/products/9789862015476?list_name=sp
https://www.tenlong.com.tw/products/9789863479659?list_name=rd
https://www.tenlong.com.tw/products/9789863479468?list_name=rd
https:/

https://www.tenlong.com.tw/orders
https://www.tenlong.com.tw/faq#category_5
https://www.tenlong.com.tw/faq#category_6
https://www.tenlong.com.tw/faq#category_7
https://www.tenlong.com.tw/faq#category_8
https://www.tenlong.com.tw/faq#category_9
https://www.tenlong.com.tw/faq#category_2
https://www.tenlong.com.tw/faq#category_1
https://www.einvoice.nat.gov.tw
https://www.tenlong.com.tw/faq#category_4
https://www.tenlong.com.tw/faq#category_3
https://www.tenlong.com.tw/products/9787111553298?list_name=sp
https://www.tenlong.com.tw/products/9786267273241?list_name=trs-t
https://www.tenlong.com.tw/products/9787121365577?list_name=rd
https://www.tenlong.com.tw/products/9787111664468?list_name=rd
https://www.tenlong.com.tw/products/9787121399336?list_name=rd
https://www.tenlong.com.tw/products/9787111666301?list_name=rd
https://www.tenlong.com.tw/products/9787111674122?list_name=rd
https://www.tenlong.com.tw/products/9787111679363?list_name=rd
https://www.tenlong.com.tw/products/9787111678441

^C
signal: interrupt


In [107]:
// Crawl3 crawls web links starting with the command-line arguments.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 243.

//
// This version uses bounded parallelism.
// For simplicity, it does not address the termination problem.
//
//package main

import (
	"fmt"
	"log"
	"os"

	"gopl.io/ch5/links"
)

func crawl(url string) []string {
	fmt.Println(url)
	list, err := links.Extract(url)
	if err != nil {
		log.Print(err)
	}
	return list
}

//!+
%args http://gopl.io/
func main() {
	worklist := make(chan []string)  // lists of URLs, may have duplicates
	unseenLinks := make(chan string) // de-duplicated URLs

	// Add command-line arguments to worklist.
	go func() { worklist <- os.Args[1:] }()

	// Create 20 crawler goroutines to fetch each unseen link.
	for i := 0; i < 20; i++ {
		go func() {
			for link := range unseenLinks {
				foundLinks := crawl(link)
				go func() { worklist <- foundLinks }()
			}
		}()
	}

	// The main goroutine de-duplicates worklist items
	// and sends the unseen ones to the crawlers.
	seen := make(map[string]bool)
	for list := range worklist {
		for _, link := range list {
			if !seen[link] {
				seen[link] = true
				unseenLinks <- link
			}
		}
	}
}

//!-

http://gopl.io/
http://www.amazon.com/dp/020161586X?tracking_id=disfordig-20
http://www.gopl.io/translations.html
http://www.gopl.io/errata.html
http://www.informit.com/store/go-programming-language-9780134190440
https://github.com/golang/tools/blob/master/refactor/eg/eg.go
http://golang.org/s/oracle-user-manual
https://github.com/golang/tools/blob/master/refactor/rename/rename.go
http://www.amazon.com/dp/0131103628?tracking_id=disfordig-20
http://www.amazon.com/dp/0134190440
http://www.barnesandnoble.com/w/1121601944
https://github.com/adonovan/gopl.io/
http://www.gopl.io/ch1.pdf
http://www.gopl.io/reviews.html
http://golang.org/lib/godoc/analysis/help.html
http://www.acornpub.co.kr/book/go-programming
https://novatec.com.br/livros/linguagem-de-programacao-go/
http://www.williamspublishing.com/Books/978-5-8459-2051-5.html
http://helion.pl/ksiazki/jezyk-go-poznaj-i-programuj-alan-a-a-donovan-brian-w-kernighan,jgopop.htm
http://helion.pl/
http://www.amazon.co.jp/exec/obidos/ASIN/4621300

2023/11/26 16:16:19 Get "http://www.pearsonapac.com/": dial tcp: lookup www.pearsonapac.com: no such host


https://www.usenix.org/system/files/login/articles/login_dec15_17_books.pdf
http://lpar.ath0.com/2015/12/03/review-go-programming-language-book
http://www.computingreviews.com/index_dynamic.cfm?CFID=15675338&CFTOKEN=37047869
http://www.infoq.com/articles/the-go-programming-language-book-review
http://www.onebigfluke.com/2016/03/book-review-go-programming-language.html
http://eli.thegreenplace.net/2016/book-review-the-go-programming-language-by-alan-donovan-and-brian-kernighan
http://www.amazon.com/Programming-Language-Addison-Wesley-Professional-Computing/product-reviews/0134190440/ref=cm_cr_dp_see_all_summary
https://github.com/golang/proposal/blob/master/design/12416-cgo-pointers.md
https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou?ie=UTF8&nodeId=508088
https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496
http://www.acornpub.co.kr/book/go-programming#main
http://www.acornpub.co.kr/
http://www.acornpub.co.kr/book/go-programmin

2023/11/26 16:16:20 getting http://www.acornpub.co.kr/download/go-programming: 403 Forbidden
2023/11/26 16:16:20 getting https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou?ie=UTF8&nodeId=508088: 403 Forbidden


http://www.acornpub.co.kr/book/go-programming#tab-submit
http://www.gopl.io/
http://www.acornpub.co.kr/series/programming-language
http://www.acornpub.co.kr/series/embedded
http://www.acornpub.co.kr/series/advanced
http://www.acornpub.co.kr/book/spring-webdev-set
http://www.acornpub.co.kr/book/9algorithms


2023/11/26 16:16:20 getting https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496: 403 Forbidden


mailto:acornpub@acornpub.co.kr
http://www.acornpub.co.kr/sitemap


2023/11/26 16:16:20 Get "mailto:acornpub@acornpub.co.kr": unsupported protocol scheme "mailto"


http://www.acornpub.co.kr/book/go-programming#header
https://www.tenlong.com.tw/
https://www.tenlong.com.tw/announcements/314
https://www.tenlong.com.tw/announcements/313
https://www.tenlong.com.tw/announcements/310
https://www.tenlong.com.tw/announcements/307
https://www.tenlong.com.tw/announcements/305
https://www.tenlong.com.tw/announcements/302
https://www.tenlong.com.tw/events/enservice
https://www.tenlong.com.tw/events/2023-bts-booklist
https://www.tenlong.com.tw/events/2023-svip
https://www.tenlong.com.tw/events/106
https://www.tenlong.com.tw/events/5g-6g
https://www.tenlong.com.tw/events/subcribe
https://www.tenlong.com.tw/categories/net
https://www.tenlong.com.tw/categories/5g
https://www.tenlong.com.tw/categories/agile
https://www.tenlong.com.tw/categories/aws
https://www.tenlong.com.tw/categories/android
https://www.tenlong.com.tw/categories/angular
https://www.tenlong.com.tw/categories/ansible
https://www.tenlong.com.tw/categories/apple-developer
https://www.tenlong.com.tw/

^C
signal: interrupt


## Multiplexing with select
* Countdown for a rocket launch
* countdown1 : time.Tick returns a channel on which it sends events periodically
* countdown2 : abort the launch sequence by pressing the return key during the countdown
* countdown3 : abort the launch sequence by pressing the return key and print the countdown

In [108]:
// Countdown1 implements the countdown for a rocket launch.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 244.


//package main

import (
	"fmt"
	"time"
)

//!+
func main() {
	fmt.Println("Commencing countdown.")
	tick := time.Tick(1 * time.Second)
	for countdown := 10; countdown > 0; countdown-- {
		fmt.Println(countdown)
		<-tick
	}
	launch()
}

//!-

func launch() {
	fmt.Println("Lift off!")
}

Commencing countdown.
10
9
8
7
6
5


signal: interrupt


In [32]:
//main()

Commencing countdown.
10
9
8
7
6
5
4
3
2
1
Lift off!


In [109]:
// Countdown2 implements the countdown for a rocket launch.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 244.

//package main

import (
	"fmt"
	"os"
	"time"
)

//!+

func main() {
	// ...create abort channel...

	//!-

	//!+abort
	abort := make(chan struct{})
	go func() {
		os.Stdin.Read(make([]byte, 1)) // read a single byte
		abort <- struct{}{}
	}()
	//!-abort

	//!+
	fmt.Println("Commencing countdown.  Press return to abort.")
	select {
	case <-time.After(10 * time.Second):
		// Do nothing.
	case <-abort:
		fmt.Println("Launch aborted!")
		return
	}
	launch()
}

//!-

func launch() {
	fmt.Println("Lift off!")
}

Commencing countdown.  Press return to abort.


^C
signal: interrupt


In [111]:
%%
ch := make(chan int,1)
for i:=0; i<10;i ++ {
    select {
    case  x := <- ch:
        fmt.Println(x) // "0" "2" "4" "6" "8"
    case  ch <- i:
    }
}

0
2
4
6
8


In [112]:
// Countdown3 implements the countdown for a rocket launch.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 246.


//package main

// NOTE: the ticker goroutine never terminates if the launch is aborted.
// This is a "goroutine leak".

import (
	"fmt"
	"os"
	"time"
)

//!+

func main() {
	// ...create abort channel...

	//!-

	abort := make(chan struct{})
	go func() {
		os.Stdin.Read(make([]byte, 1)) // read a single byte
		abort <- struct{}{}
	}()

	//!+
	fmt.Println("Commencing countdown.  Press return to abort.")
	tick := time.Tick(1 * time.Second)
	for countdown := 10; countdown > 0; countdown-- {
		fmt.Println(countdown)
		select {
		case <-tick:
			// Do nothing.
		case <-abort:
			fmt.Println("Launch aborted!")
			return
		}
	}
	launch()
}

//!-

func launch() {
	fmt.Println("Lift off!")
}

Commencing countdown.  Press return to abort.
10
9
8
7
6


^C
signal: interrupt


## Example: Concurrent Directory Travsal
* du1: disk usage without parallelism 
* du2: du1 with progress every 500ms
* du3: walkDir concurrently 

In [114]:
// The du1 command computes the disk usage of the files in a directory.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 247.

//!+main

//package main

import (
	"flag"
	"fmt"
	"io/ioutil"
	"os"
	"path/filepath"
)
%args /etc
func main() {
	// Determine the initial directories.
	flag.Parse()
	roots := flag.Args()
	if len(roots) == 0 {
		roots = []string{"."}
	}

	// Traverse the file tree.
	fileSizes := make(chan int64)
	go func() {
		for _, root := range roots {
			walkDir(root, fileSizes)
		}
		close(fileSizes)
	}()

	// Print the results.
	var nfiles, nbytes int64
	for size := range fileSizes {
		nfiles++
		nbytes += size
	}
	printDiskUsage(nfiles, nbytes)
}

func printDiskUsage(nfiles, nbytes int64) {
	fmt.Printf("%d files  %.1f GB\n", nfiles, float64(nbytes)/1e9)
}

//!-main

//!+walkDir

// walkDir recursively walks the file tree rooted at dir
// and sends the size of each found file on fileSizes.
func walkDir(dir string, fileSizes chan<- int64) {
	for _, entry := range dirents(dir) {
		if entry.IsDir() {
			subdir := filepath.Join(dir, entry.Name())
			walkDir(subdir, fileSizes)
		} else {
			fileSizes <- entry.Size()
		}
	}
}

// dirents returns the entries of directory dir.
func dirents(dir string) []os.FileInfo {
	entries, err := ioutil.ReadDir(dir)
	if err != nil {
		fmt.Fprintf(os.Stderr, "du1: %v\n", err)
		return nil
	}
	return entries
}

//!-walkDir

// The du1 variant uses two goroutines and
// prints the total after every file is found.

du1: open /etc/cups/certs: permission denied


243 files  0.0 GB


In [38]:
os.Args = []string{"du1","/etc"}
main()

du1: open /etc/polkit-1/localauthority: permission denied
du1: open /etc/ssl/private: permission denied


1798 files  0.0 GB


In [115]:
// The du2 command computes the disk usage of the files in a directory.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 249.

//package main

// The du2 variant uses select and a time.Ticker
// to print the totals periodically if -v is set.

import (
	"flag"
	"fmt"
	"io/ioutil"
	"os"
	"path/filepath"
	"time"
)

//!+
var verbose = flag.Bool("v", false, "show verbose progress messages")
%args /etc
func main() {
	// ...start background goroutine...

	//!-
	// Determine the initial directories.
	flag.Parse()
	roots := flag.Args()
	if len(roots) == 0 {
		roots = []string{"."}
	}

	// Traverse the file tree.
	fileSizes := make(chan int64)
	go func() {
		for _, root := range roots {
			walkDir(root, fileSizes)
		}
		close(fileSizes)
	}()

	//!+
	// Print the results periodically.
	var tick <-chan time.Time
	if *verbose {
		tick = time.Tick(500 * time.Millisecond)
	}
	var nfiles, nbytes int64
loop:
	for {
		select {
		case size, ok := <-fileSizes:
			if !ok {
				break loop // fileSizes was closed
			}
			nfiles++
			nbytes += size
		case <-tick:
			printDiskUsage(nfiles, nbytes)
		}
	}
	printDiskUsage(nfiles, nbytes) // final totals
}

//!-

func printDiskUsage(nfiles, nbytes int64) {
	fmt.Printf("%d files  %.1f GB\n", nfiles, float64(nbytes)/1e9)
}

// walkDir recursively walks the file tree rooted at dir
// and sends the size of each found file on fileSizes.
func walkDir(dir string, fileSizes chan<- int64) {
	for _, entry := range dirents(dir) {
		if entry.IsDir() {
			subdir := filepath.Join(dir, entry.Name())
			walkDir(subdir, fileSizes)
		} else {
			fileSizes <- entry.Size()
		}
	}
}

// dirents returns the entries of directory dir.
func dirents(dir string) []os.FileInfo {
	entries, err := ioutil.ReadDir(dir)
	if err != nil {
		fmt.Fprintf(os.Stderr, "du: %v\n", err)
		return nil
	}
	return entries
}

du: open /etc/cups/certs: permission denied


243 files  0.0 GB


In [1]:
// The du3 command computes the disk usage of the files in a directory
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 250.

//package main

// The du3 variant traverses all directories in parallel.
// It uses a concurrency-limiting counting semaphore
// to avoid opening too many files at once.

import (
	"flag"
	"fmt"
	"io/ioutil"
	"os"
	"path/filepath"
	"sync"
	"time"
)

var vFlag = flag.Bool("v", false, "show verbose progress messages")
%args -v /etc
//!+
func main() {
	// ...determine roots...

	//!-
	flag.Parse()

	// Determine the initial directories.
	roots := flag.Args()
	if len(roots) == 0 {
		roots = []string{"."}
	}

	//!+
	// Traverse each root of the file tree in parallel.
	fileSizes := make(chan int64)
	var n sync.WaitGroup
	for _, root := range roots {
		n.Add(1)
		go walkDir(root, &n, fileSizes)
	}
	go func() {
		n.Wait()
		close(fileSizes)
	}()
	//!-

	// Print the results periodically.
	var tick <-chan time.Time
	if *vFlag {
		tick = time.Tick(500 * time.Millisecond)
	}
	var nfiles, nbytes int64
loop:
	for {
		select {
		case size, ok := <-fileSizes:
			if !ok {
				break loop // fileSizes was closed
			}
			nfiles++
			nbytes += size
		case <-tick:
			printDiskUsage(nfiles, nbytes)
		}
	}

	printDiskUsage(nfiles, nbytes) // final totals
	//!+
	// ...select loop...
}

//!-

func printDiskUsage(nfiles, nbytes int64) {
	fmt.Printf("%d files  %.1f GB\n", nfiles, float64(nbytes)/1e9)
}

// walkDir recursively walks the file tree rooted at dir
// and sends the size of each found file on fileSizes.
//!+walkDir
func walkDir(dir string, n *sync.WaitGroup, fileSizes chan<- int64) {
	defer n.Done()
	for _, entry := range dirents(dir) {
		if entry.IsDir() {
			n.Add(1)
			subdir := filepath.Join(dir, entry.Name())
			go walkDir(subdir, n, fileSizes)
		} else {
			fileSizes <- entry.Size()
		}
	}
}

//!-walkDir

//!+sema
// sema is a counting semaphore for limiting concurrency in dirents.
var sema = make(chan struct{}, 20)

// dirents returns the entries of directory dir.
func dirents(dir string) []os.FileInfo {
	sema <- struct{}{}        // acquire token
	defer func() { <-sema }() // release token
	// ...
	//!-sema

	entries, err := ioutil.ReadDir(dir)
	if err != nil {
		fmt.Fprintf(os.Stderr, "du: %v\n", err)
		return nil
	}
	return entries
}

du: open /etc/cups/certs: permission denied


243 files  0.0 GB


## Cancellation

In [2]:
// The du4 command computes the disk usage of the files in a directory.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 251.

//package main

// The du4 variant includes cancellation:
// it terminates quickly when the user hits return.

import (
	"fmt"
	"os"
	"path/filepath"
	"sync"
	"time"
)

//!+1
var done = make(chan struct{})

func cancelled() bool {
	select {
	case <-done:
		return true
	default:
		return false
	}
}

In [3]:
// du4, main
%args /etc
func main() {
	// Determine the initial directories.
	roots := os.Args[1:]
	if len(roots) == 0 {
		roots = []string{"."}
	}

	//!+2
	// Cancel traversal when input is detected.
	go func() {
		os.Stdin.Read(make([]byte, 1)) // read a single byte
		close(done)
	}()
	//!-2

	// Traverse each root of the file tree in parallel.
	fileSizes := make(chan int64)
	var n sync.WaitGroup
	for _, root := range roots {
		n.Add(1)
		go walkDir(root, &n, fileSizes)
	}
	go func() {
		n.Wait()
		close(fileSizes)
	}()

	// Print the results periodically.
	tick := time.Tick(500 * time.Millisecond)
	var nfiles, nbytes int64
loop:
	//!+3
	for {
		select {
		case <-done:
			// Drain fileSizes to allow existing goroutines to finish.
			for range fileSizes {
				// Do nothing.
			}
			return
		case size, ok := <-fileSizes:
			// ...
			//!-3
			if !ok {
				break loop // fileSizes was closed
			}
			nfiles++
			nbytes += size
		case <-tick:
			printDiskUsage(nfiles, nbytes)
		}
	}
	printDiskUsage(nfiles, nbytes) // final totals
}



du: open /etc/cups/certs: permission denied


243 files  0.0 GB


In [4]:
//printDiskUsage, walkDir, dirents
func printDiskUsage(nfiles, nbytes int64) {
	fmt.Printf("%d files  %.1f GB\n", nfiles, float64(nbytes)/1e9)
}

// walkDir recursively walks the file tree rooted at dir
// and sends the size of each found file on fileSizes.
//!+4
func walkDir(dir string, n *sync.WaitGroup, fileSizes chan<- int64) {
	defer n.Done()
	if cancelled() {
		return
	}
	for _, entry := range dirents(dir) {
		// ...
		//!-4
		if entry.IsDir() {
			n.Add(1)
			subdir := filepath.Join(dir, entry.Name())
			go walkDir(subdir, n, fileSizes)
		} else {
			fileSizes <- entry.Size()
		}
		//!+4
	}
}

//!-4

var sema = make(chan struct{}, 20) // concurrency-limiting counting semaphore

// dirents returns the entries of directory dir.
//!+5
func dirents(dir string) []os.FileInfo {
	select {
	case sema <- struct{}{}: // acquire token
	case <-done:
		return nil // cancelled
	}
	defer func() { <-sema }() // release token

	// ...read directory...
	//!-5

	f, err := os.Open(dir)
	if err != nil {
		fmt.Fprintf(os.Stderr, "du: %v\n", err)
		return nil
	}
	defer f.Close()

	entries, err := f.Readdir(0) // 0 => no limit; read all entries
	if err != nil {
		fmt.Fprintf(os.Stderr, "du: %v\n", err)
		// Don't return: Readdir may return partial results.
	}
	return entries
}

## Example: Chat Server

In [5]:
// broadcaster
// Chat is a server that lets clients chat with each other.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 254.
//!+

//package main

import (
	"bufio"
	"fmt"
	"log"
	"net"
)

//!+broadcaster
type client chan<- string // an outgoing message channel

var (
	entering = make(chan client)
	leaving  = make(chan client)
	messages = make(chan string) // all incoming client messages
)

func broadcaster() {
	clients := make(map[client]bool) // all connected clients
	for {
		select {
		case msg := <-messages:
			// Broadcast incoming message to all
			// clients' outgoing message channels.
			for cli := range clients {
				cli <- msg
			}

		case cli := <-entering:
			clients[cli] = true

		case cli := <-leaving:
			delete(clients, cli)
			close(cli)
		}
	}
}



In [8]:
//!+handleConn
//clientWriter and main
func clientWriter(conn net.Conn, ch <-chan string) {
	for msg := range ch {
		fmt.Fprintln(conn, msg) // NOTE: ignoring network errors
	}
}
func handleConn(conn net.Conn) {
	ch := make(chan string) // outgoing client messages
	go clientWriter(conn, ch)

	who := conn.RemoteAddr().String()
	ch <- "You are " + who
	messages <- who + " has arrived"
	entering <- ch

	input := bufio.NewScanner(conn)
	for input.Scan() {
		messages <- who + ": " + input.Text()
	}
	// NOTE: ignoring potential errors from input.Err()

	leaving <- ch
	messages <- who + " has left"
	conn.Close()
}



In [9]:


//!-handleConn

//!+main
func main() {
	listener, err := net.Listen("tcp", "localhost:8000")
	if err != nil {
		log.Fatal(err)
	}

	go broadcaster()
	for {
		conn, err := listener.Accept()
		if err != nil {
			log.Print(err)
			continue
		}
		go handleConn(conn)
	}
}

//!-main

^C
signal: interrupt
